# Question B4 (10 marks)

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install pytorch_tabular[extra]
!pip install alibi-detect --user

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

1.Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
df = df.drop(['full_address', 'nearest_stn'], axis=1)
train = df[df.year < 2020]
val = df[df.year == 2020]
test_2022 = df[df.year == 2022]
test_2023 = df[df.year == 2023]

In [9]:
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular import TabularModel

continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_cols = [ "month", "town", "flat_model_type", "storey_range"]

data_config_2022 = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config_2022 = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2022 = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2022 = OptimizerConfig()

tab_model_2022 = TabularModel(
    data_config=data_config_2022,
    model_config=model_config_2022,
    optimizer_config=optimizer_2022,
    trainer_config=trainer_config_2022,
)

tab_model_2022.fit(train=train, validation=val)
result_2022 = tab_model_2022.evaluate(test_2022)
pred_df_2022 = tab_model_2022.predict(test_2022)


2024-04-21 09:57:15,845 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-04-21 09:57:15,913 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-04-21 09:57:15,947 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-04-21 09:57:16,203 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: CategoryEmbeddingModel

...

2024-04-21 09:57:52,818 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-04-21 09:57:52,822 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

Output()

C:\Users\nsupr\anaconda3\envs\tensorflow\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       16876774400.0       │
│  test_mean_squared_error  │       16876774400.0       │
└───────────────────────────┴───────────────────────────┘

In [10]:
from sklearn.metrics import r2_score

print(f"{result_2022[0]['test_mean_squared_error']}")
mse = np.sqrt(float(result_2022[0]['test_mean_squared_error']))

pred_2022 = pred_df_2022["resale_price_prediction"].values.tolist()
y_2022 = pred_df_2022['resale_price'].values.tolist()

rsquared = r2_score(y_2022, pred_2022)
print(f"test loss: {result_2022[0]['test_loss']}")
print(f"RMSE: {mse}")
print(f"R Squared: {rsquared}")

16876774400.0


KeyError: 'resale_price'

2.Evaluate your model from B1 on data from year 2023 and report the test R2.

In [ ]:
# TODO: Enter your code here

3.Did model degradation occur for the deep learning model?


In [ ]:
# YOUR ANSWER HERE



---



---



4.Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [ ]:
# YOUR CODE HERE

5.Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


In [ ]:
# YOUR ANSWER HERE

6.From your analysis via TabularDrift, which features contribute to this shift?


In [ ]:
# YOUR ANSWER HERE

7.Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


In [ ]:
# YOUR CODE HERE

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |